In [86]:
datapath = "/home/wbm001/DeepLPI/DeepLPI/data/"
datafile = '/home/wbm001/DeepLPI/DeepLPI/data/mol_dict.csv'
modelpath = "/home/wbm001/DeepLPI/data/saved_models/model_300dim.pkl"

In [113]:
from rdkit.Chem import SDMolSupplier
from rdkit.Chem.Draw import IPythonConsole

import pandas as pd

data = pd.read_csv(datafile)

In [114]:
data = data[["0"]]
data

,0
0,Cc1ccc(c(F)c1Oc1ncccn1)-c1cnc(N)cn1
1,CC(Oc1ccccc1)C(=O)NCB(O)O
2,COc1cc(CCNCCCc2ccccc2)c(OC)cc1Br
3,COC(=O)C1CCC2(CCN(C2=O)c2ccc(cc2)[C@H]2CCC(C2)...
4,O=C1NC(=O)N(C1c1ccc(cc1)-c1nnn(CC2CC2)n1)c1ccc...
...,...
230328,O=c1c(-c2ccccc2)c2ccccn2c(=O)n1CCCCN1CCCC(C1)c...
230329,COC(=O)c1cccnc1-c1cnc(o1)C(=O)CCCCCCc1ccccc1
230330,Cc1cc(Nc2cc3nc(CN4CCC(F)(F)CC4)ccc3c(N[C@H]3C[...
230331,CCCN(CC1CC1)c1nc(C)nc2n(-c3c(C)cc(C)cc3C)c(=S)...


In [115]:
from mol2vec.features import mol2alt_sentence, MolSentence, DfVec, sentences2vec
from gensim.models import Word2Vec
from rdkit.Chem import MolFromSmiles as sm
import numpy as np

data["sent"] = data["0"].map(lambda x : sm(x))

RDKit ERROR: [07:20:35] Explicit valence for atom # 1 B, 6, is greater than permitted
RDKit ERROR: [07:20:35] Explicit valence for atom # 24 N, 4, is greater than permitted
RDKit ERROR: [07:20:35] Explicit valence for atom # 2 N, 4, is greater than permitted
RDKit ERROR: [07:20:35] Can't kekulize mol.  Unkekulized atoms: 11 13 14 15 16
RDKit ERROR: 
RDKit ERROR: [07:20:35] Can't kekulize mol.  Unkekulized atoms: 1 2 27
RDKit ERROR: 
RDKit ERROR: [07:20:36] Can't kekulize mol.  Unkekulized atoms: 1 2 3 14 16
RDKit ERROR: 
RDKit ERROR: [07:20:36] Explicit valence for atom # 31 N, 4, is greater than permitted
RDKit ERROR: [07:20:36] Can't kekulize mol.  Unkekulized atoms: 13 14 18
RDKit ERROR: 
RDKit ERROR: [07:20:36] Can't kekulize mol.  Unkekulized atoms: 12 13 14 15 16 19 20 21 22
RDKit ERROR: 
RDKit ERROR: [07:20:36] Explicit valence for atom # 8 C, 6, is greater than permitted
RDKit ERROR: [07:20:36] Can't kekulize mol.  Unkekulized atoms: 5 6 7 8 9 10 11 12 13 17 18 19 30 31 32
RDKi

In [132]:
def mol2alt(x):
    try:
        re = mol2alt_sentence(x,1)
    except:
        re = None
    return re

In [134]:
data_sentence = data["sent"].map(lambda x : mol2alt(x))

In [135]:
from gensim.models import word2vec

model = word2vec.Word2Vec.load(modelpath)

In [147]:
data_sentence.dropna()

0         [2246728737, 422715066, 3217380708, 3124581743...
1         [2246728737, 3537119515, 2245273601, 162628120...
2         [2246728737, 3975275337, 864674487, 2076190208...
3         [2246728737, 3975275337, 864674487, 2154935424...
4         [864942730, 10565946, 3217380708, 3182177290, ...
                                ...                        
230328    [864942730, 10565946, 3217380708, 2370996728, ...
230329    [2246728737, 3975275337, 864674487, 2154935424...
230330    [2246728737, 422715066, 3217380708, 1307265339...
230331    [2246728737, 3542456614, 2245384272, 117312591...
230332    [2246728737, 3545365497, 2246699815, 283456642...
Name: sent, Length: 230173, dtype: object

In [148]:
molvec = sentences2vec(data_sentence.dropna(),model,unseen='UNK')

In [159]:
np.array(data_sentence.dropna().index).reshape(-1,1)

array([[     0],
       [     1],
       [     2],
       ...,
       [230330],
       [230331],
       [230332]])

In [163]:
molembed = pd.DataFrame(np.concatenate([np.array(data_sentence.dropna().index).reshape(-1,1),molvec],axis=1))

In [167]:
molembed[0] = molembed[0].astype(int)
molembed = molembed.set_index(0)

In [181]:
molembed.to_csv(datapath+"mol_embed.csv")